## Overview

- **Expectation Used:** [expect_profile_numeric_columns_diff_between_inclusive_threshold_range](https://github.com/great-expectations/great_expectations/blob/develop/contrib/capitalone_dataprofiler_expectations/capitalone_dataprofiler_expectations/expectations/expect_profile_numeric_columns_diff_between_inclusive_threshold_range.py)

- **Expectation Description:** This expectation will check the user-specified difference between user-specified metrics from an original report generated by the Data Profiler and a new report. The two reports are generated on two different sets of data with matching schemas.

- **Example Details:** In this example a data owner has a dataset containing salary information on individuals in the data science field ranging from **2020** to **2022**. The data owner has been tasked with checking the historical trends of salary growth across the industry with the numbers they have. They want to use this expectation to determine if the median growth of salaries at their company are within the same range of median growth as the companies they have data on.

### Imports

In [ ]:
import os

import pandas as pd
import numpy as np

# Great expectations imports
import great_expectations as ge
from capitalone_dataprofiler_expectations.expectations. \
    expect_profile_numeric_columns_diff_between_inclusive_threshold_range \
    import ExpectProfileNumericColumnsDiffBetweenInclusiveThresholdRange
from great_expectations.self_check.util import build_pandas_validator_with_data

# Data Profiler import
import dataprofiler as dp

### Setup
Below we are going to import a dataset from the Data Profile testing suite. This csv holds information on the salaries of individuals in the data science field from all over the world.

In [ ]:
context = ge.get_context()

In [ ]:
data_path = "../../dataprofiler/tests/data/csv/ds_salaries.csv"
data = dp.Data(data_path).data
data

For this expectation we are going to compare the diff median `salary_in_usd` between `work_year` **2020** and **2022**. Below we are gathering the different years that are recorded in this dataset.

In [ ]:
data.sort_values(by="work_year", axis=0, inplace=True)
years = data["work_year"].unique().tolist()
years

Now that we have the years, we will capture all records from each year in their own dataframes so that we can process them separately.

In [ ]:
individual_dataframes = []
for year in years:
    current_year_df = data.loc[data["work_year"]==year]
    current_year_df = current_year_df.drop("work_year", axis=1)
    individual_dataframes.append(current_year_df)
year_2020_salary_df = individual_dataframes[0]
year_2022_salary_df = individual_dataframes[2]

Now we will create a report on the first individual dataframe which corresponds to the year **2020**, then we will output the median `salary_in_usd` from this dataframe.

In [ ]:
profiler_options = dp.ProfilerOptions()
profiler_options.set({"data_labeler.is_enabled": False})

profile = dp.Profiler(year_2020_salary_df, len(year_2020_salary_df), options=profiler_options)
profile.save(filepath='year_2020_salary_profile.pkl')
report  = profile.report(report_options={"output_format": "compact"})

Let's take a look at the output `median` metric from the `salary_in_usd` column as found in the report

In [ ]:
report['data_stats'][6]['statistics']['median']

The data owner has been told by his company that median salary increases were about **10000** from **2020** to **2022**. So the data owner wants to for their range around this number. With the validator below, an expectation is set up with a difference between the `median` `salary_in_usd` of at least **7500** and no more than **12500**. Meaning, this validator will check if the `median` `salary_in_usd` has increased by **7500** to **12500** from **2020** to **2022** across all the companies they have data on.

In [ ]:
validator = build_pandas_validator_with_data(year_2022_salary_df)
results = validator.expect_profile_numeric_columns_diff_between_inclusive_threshold_range(
    profile_path='year_2020_salary_profile.pkl',
    limit_check_report_keys={
            "salary_in_usd": {
                "median": {"lower": 7500, "upper": 12500},
            },
        }
)

### Results
From the output below, the data owner can see that the expectation has an unexpected value. The result shows that the diff between the two profiles is significantly more than the upperbound at about **44000**. This indicates to the data owner that the salary growth trends in competing companies have increased more than his own company in the last two years.

In [ ]:
results